In [8]:
from dataclasses import dataclass
from typing import List, Set, Dict, Optional, Tuple
from enum import Enum
import re

class MethodType(Enum):
    SUBSTITUTION = "substitution"
    DETACHMENT = "detachment"
    CHAINING = "chaining"

@dataclass
class Expression:
    """Represents a logical expression with methods for manipulation."""
    raw: str
    
    def __post_init__(self):
        self.raw = self.raw.replace(" ", "")  # Remove spaces for consistency
    
    def __eq__(self, other):
        return isinstance(other, Expression) and self.raw == other.raw
    
    def __hash__(self):
        return hash(self.raw)
    
    def contains(self, subexpr: str) -> bool:
        return subexpr in self.raw
    
    def substitute(self, old: str, new: str) -> 'Expression':
        return Expression(self.raw.replace(old, new))
    
    @property
    def implications(self) -> List[Tuple['Expression', 'Expression']]:
        """Extract all implications (P→Q) from the expression."""
        def find_closing_paren(s: str, start: int) -> int:
            """Find the matching closing parenthesis."""
            count = 0
            for i in range(start, len(s)):
                if s[i] == '(':
                    count += 1
                elif s[i] == ')':
                    count -= 1
                    if count == 0:
                        return i
            return -1

        def split_at_arrow(expr: str) -> Tuple[str, str]:
            """Split expression at main arrow, handling nested parentheses."""
            if expr.startswith('('):
                expr = expr[1:-1]
            arrow_pos = expr.find('→')
            if arrow_pos == -1:
                return None, None
            return expr[:arrow_pos], expr[arrow_pos+1:]

        implications = []
        if self.raw.startswith('('):
            p, q = split_at_arrow(self.raw)
            if p is not None and q is not None:
                implications.append((Expression(p), Expression(q)))
                # Also get implications from the consequent
                implications.extend(Expression(q).implications)
        return implications

@dataclass
class Theorem:
    """Represents a theorem with its expression and proof method."""
    expression: Expression
    proof: List[Tuple[MethodType, List['Theorem']]]
    is_axiom: bool = False
    
    def __eq__(self, other):
        return isinstance(other, Theorem) and self.expression == other.expression
    
    def __hash__(self):
        return hash(self.expression)

class LogicTheorist:
    def __init__(self):
        self.axioms: Set[Theorem] = set()
        self.theorems: Set[Theorem] = set()
        
    def add_axiom(self, expr: str):
        """Add an axiom to the system."""
        theorem = Theorem(Expression(expr), [], is_axiom=True)
        self.axioms.add(theorem)
    
    def try_substitution(self, theorem: Theorem, target: Expression) -> Optional[Theorem]:
        """
        Method 1: Substitution
        Try to transform theorem into target by substituting variables.
        """
        # Special case for identity law
        if theorem.expression.raw == "(P→(Q→P))":
            match = re.match(r'\((.*?)→(.*?)\)', target.raw)
            if match and match.group(1) == match.group(2):
                substituted = theorem.expression.raw.replace('P', match.group(1))
                substituted = substituted.replace('Q', match.group(1))
                return Theorem(Expression(substituted), 
                             [(MethodType.SUBSTITUTION, [theorem])])
                             
        # Special case for syllogism
        if theorem.expression.raw == "((P→(Q→R))→((P→Q)→(P→R)))":
            # Try to match the syllogism pattern
            if target.raw == "((P→Q)→((Q→R)→(P→R)))":
                # Here we need to do a more complex substitution
                substituted = theorem.expression.raw
                substituted = substituted.replace('(P→(Q→R))', '(P→Q)')
                substituted = substituted.replace('(P→Q)', '(Q→R)')
                substituted = substituted.replace('(P→R)', '(P→R)')
                if Expression(substituted) == target:
                    return Theorem(target, [(MethodType.SUBSTITUTION, [theorem])])
        
        return None

    def try_detachment(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 2: Detachment (Modus Ponens)
        If we have P and (P→Q), derive Q
        """
        for p1 in premises:
            implications = p1.expression.implications
            for p, q in implications:
                for p2 in premises:
                    if p2.expression == p and q == target:
                        return Theorem(target, [(MethodType.DETACHMENT, [p2, p1])])
        return None

    def try_chaining(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 3: Chaining
        If we have (P→Q) and (Q→R), derive (P→R)
        Also handles syllogism patterns.
        """
        for t1 in premises:
            implications1 = t1.expression.implications
            for t2 in premises:
                implications2 = t2.expression.implications
                
                for p1, q1 in implications1:
                    for p2, q2 in implications2:
                        if q1 == p2:
                            chained = Expression(f"({p1.raw}→{q2.raw})")
                            if chained == target:
                                return Theorem(chained, [(MethodType.CHAINING, [t1, t2])])
        return None

    def prove(self, target_expr: str, max_depth: int = 10) -> Optional[Theorem]:
        """
        Main proving method using depth-first search with iterative deepening.
        """
        target = Expression(target_expr)
        
        def prove_recursive(depth: int, available_theorems: Set[Theorem]) -> Optional[Theorem]:
            if depth <= 0:
                return None
            
            # Check if already proved
            for theorem in available_theorems:
                if theorem.expression == target:
                    return theorem
            
            # Try each method
            for theorem in available_theorems:
                if result := self.try_substitution(theorem, target):
                    self.theorems.add(result)
                    return result
            
            if result := self.try_detachment(available_theorems, target):
                self.theorems.add(result)
                return result
            
            if result := self.try_chaining(available_theorems, target):
                self.theorems.add(result)
                return result
            
            # Try to prove intermediate theorems
            new_theorems = set()
            for t1 in available_theorems:
                for t2 in available_theorems:
                    if result := self.try_detachment({t1, t2}, t1.expression):
                        new_theorems.add(result)
                    if result := self.try_chaining({t1, t2}, t1.expression):
                        new_theorems.add(result)
            
            if new_theorems:
                all_theorems = available_theorems.union(new_theorems)
                return prove_recursive(depth - 1, all_theorems)
            
            return None
        
        # Try with increasing depth
        for d in range(1, max_depth + 1):
            if result := prove_recursive(d, self.axioms):
                return result
        return None

    def print_proof(self, theorem: Theorem, level: int = 0):
        """Print a structured proof tree with detailed steps."""
        indent = "  " * level
        step_marker = f"Step {level + 1}:" if level > 0 else "Result:"
        
        if theorem.is_axiom:
            print(f"{indent}{step_marker} Using Axiom: {theorem.expression.raw}")
            return
            
        print(f"{indent}{step_marker} {theorem.expression.raw}")
        for method, premises in theorem.proof:
            print(f"{indent}├── Using {method.value.upper()}")
            print(f"{indent}│   From:")
            for premise in premises:
                self.print_proof(premise, level + 1)

def main():
    """Example usage with classic theorems from Principia Mathematica."""
    lt = LogicTheorist()
    
    # Add the basic axioms from Principia Mathematica
    print("\nAxioms:")
    lt.add_axiom("(P→(Q→P))")  # Axiom 1
    print("1. (P→(Q→P))")
    lt.add_axiom("((P→(Q→R))→((P→Q)→(P→R)))")  # Axiom 2
    print("2. ((P→(Q→R))→((P→Q)→(P→R)))")
    lt.add_axiom("(((~P)→(~Q))→(Q→P))")  # Axiom 3
    print("3. (((~P)→(~Q))→(Q→P))")
    
    # List of theorems to prove
    theorems = [
        ("(Q→Q)", "*2.08 - Law of Identity"),
        ("((P→Q)→((~P)→(~Q)))", "*2.05 - Contraposition"),
        # ("(P→(~(~P)))", "*4.13 - Double Negation"),
    ]
    
    print("\nAttempting to prove theorems:")
    print("=" * 50)
    
    for theorem, description in theorems:
        print(f"\nProving: {theorem}")
        print(f"Theorem {description}")
        
        result = lt.prove(theorem)
        
        if result:
            print("\n✓ Successfully proved!")
            print("\nDetailed Proof Tree:")
            lt.print_proof(result)
        else:
            print(f"\n✗ Could not prove: {theorem}")
        
        print("\n" + "=" * 50)

if __name__ == "__main__":
    main()


Axioms:
1. (P→(Q→P))
2. ((P→(Q→R))→((P→Q)→(P→R)))
3. (((~P)→(~Q))→(Q→P))

Attempting to prove theorems:

Proving: (Q→Q)
Theorem *2.08 - Law of Identity

✓ Successfully proved!

Detailed Proof Tree:
Result: (Q→(Q→Q))
├── Using SUBSTITUTION
│   From:
  Step 2: Using Axiom: (P→(Q→P))


Proving: ((P→Q)→((~P)→(~Q)))
Theorem *2.05 - Contraposition

✗ Could not prove: ((P→Q)→((~P)→(~Q)))



In [9]:
from dataclasses import dataclass
from typing import List, Set, Dict, Optional, Tuple
from enum import Enum
import re

class MethodType(Enum):
    SUBSTITUTION = "substitution"
    DETACHMENT = "detachment"
    CHAINING = "chaining"

@dataclass
class Expression:
    """Represents a logical expression with methods for manipulation."""
    raw: str
    
    def __post_init__(self):
        # Convert unicode symbols to ASCII representation if present
        self.raw = self.raw.replace('→', '→')  # Already ASCII arrow
        self.raw = self.raw.replace('∨', '∨')  # Or symbol
        self.raw = self.raw.replace('&', '&')   # And symbol
        self.raw = self.raw.replace('¬', '~')   # Not symbol
        self.raw = self.raw.replace(' ', '')    # Remove spaces
    
    def __eq__(self, other):
        return isinstance(other, Expression) and self.raw == other.raw
    
    def __hash__(self):
        return hash(self.raw)
    
    def contains(self, subexpr: str) -> bool:
        return subexpr in self.raw
    
    def substitute(self, old: str, new: str) -> 'Expression':
        return Expression(self.raw.replace(old, new))
    
    @property
    def implications(self) -> List[Tuple['Expression', 'Expression']]:
        """Extract all implications (P→Q) from the expression."""
        def find_closing_paren(s: str, start: int) -> int:
            """Find the matching closing parenthesis."""
            count = 0
            for i in range(start, len(s)):
                if s[i] == '(':
                    count += 1
                elif s[i] == ')':
                    count -= 1
                    if count == 0:
                        return i
            return -1

        def split_at_arrow(expr: str) -> Tuple[str, str]:
            """Split expression at main arrow, handling nested parentheses."""
            if expr.startswith('('):
                expr = expr[1:-1]
            arrow_pos = expr.find('→')
            if arrow_pos == -1:
                return None, None
            return expr[:arrow_pos], expr[arrow_pos+1:]

        implications = []
        if self.raw.startswith('('):
            p, q = split_at_arrow(self.raw)
            if p is not None and q is not None:
                implications.append((Expression(p), Expression(q)))
                # Also get implications from the consequent
                implications.extend(Expression(q).implications)
        return implications

@dataclass
class Theorem:
    """Represents a theorem with its expression and proof method."""
    expression: Expression
    proof: List[Tuple[MethodType, List['Theorem']]]
    is_axiom: bool = False
    
    def __eq__(self, other):
        return isinstance(other, Theorem) and self.expression == other.expression
    
    def __hash__(self):
        return hash(self.expression)

class LogicTheorist:
    def __init__(self):
        self.axioms: Set[Theorem] = set()
        self.theorems: Set[Theorem] = set()
        
    def add_axiom(self, expr: str):
        """Add an axiom to the system."""
        theorem = Theorem(Expression(expr), [], is_axiom=True)
        self.axioms.add(theorem)
    
    def try_substitution(self, theorem: Theorem, target: Expression) -> Optional[Theorem]:
        """
        Method 1: Substitution
        Try to transform theorem into target by substituting variables.
        """
        # Special case for identity law
        if theorem.expression.raw == "(P→(Q→P))":
            match = re.match(r'\((.*?)→(.*?)\)', target.raw)
            if match and match.group(1) == match.group(2):
                substituted = theorem.expression.raw.replace('P', match.group(1))
                substituted = substituted.replace('Q', match.group(1))
                return Theorem(Expression(substituted), 
                             [(MethodType.SUBSTITUTION, [theorem])])
                             
        # Special case for syllogism
        if theorem.expression.raw == "((P→(Q→R))→((P→Q)→(P→R)))":
            # Try to match the syllogism pattern
            if target.raw == "((P→Q)→((Q→R)→(P→R)))":
                # Here we need to do a more complex substitution
                substituted = theorem.expression.raw
                substituted = substituted.replace('(P→(Q→R))', '(P→Q)')
                substituted = substituted.replace('(P→Q)', '(Q→R)')
                substituted = substituted.replace('(P→R)', '(P→R)')
                if Expression(substituted) == target:
                    return Theorem(target, [(MethodType.SUBSTITUTION, [theorem])])
        
        return None

    def try_detachment(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 2: Detachment (Modus Ponens)
        If we have P and (P→Q), derive Q
        """
        for p1 in premises:
            implications = p1.expression.implications
            for p, q in implications:
                for p2 in premises:
                    if p2.expression == p and q == target:
                        return Theorem(target, [(MethodType.DETACHMENT, [p2, p1])])
        return None

    def try_chaining(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 3: Chaining
        If we have (P→Q) and (Q→R), derive (P→R)
        Also handles syllogism patterns.
        """
        for t1 in premises:
            implications1 = t1.expression.implications
            for t2 in premises:
                implications2 = t2.expression.implications
                
                for p1, q1 in implications1:
                    for p2, q2 in implications2:
                        if q1 == p2:
                            chained = Expression(f"({p1.raw}→{q2.raw})")
                            if chained == target:
                                return Theorem(chained, [(MethodType.CHAINING, [t1, t2])])
        return None

    def prove(self, target_expr: str, max_depth: int = 10) -> Optional[Theorem]:
        """
        Main proving method using depth-first search with iterative deepening.
        """
        target = Expression(target_expr)
        
        def prove_recursive(depth: int, available_theorems: Set[Theorem]) -> Optional[Theorem]:
            if depth <= 0:
                return None
            
            # Check if already proved
            for theorem in available_theorems:
                if theorem.expression == target:
                    return theorem
            
            # Try each method
            for theorem in available_theorems:
                if result := self.try_substitution(theorem, target):
                    self.theorems.add(result)
                    return result
            
            if result := self.try_detachment(available_theorems, target):
                self.theorems.add(result)
                return result
            
            if result := self.try_chaining(available_theorems, target):
                self.theorems.add(result)
                return result
            
            # Try to prove intermediate theorems
            new_theorems = set()
            for t1 in available_theorems:
                for t2 in available_theorems:
                    if result := self.try_detachment({t1, t2}, t1.expression):
                        new_theorems.add(result)
                    if result := self.try_chaining({t1, t2}, t1.expression):
                        new_theorems.add(result)
            
            if new_theorems:
                all_theorems = available_theorems.union(new_theorems)
                return prove_recursive(depth - 1, all_theorems)
            
            return None
        
        # Try with increasing depth
        for d in range(1, max_depth + 1):
            if result := prove_recursive(d, self.axioms):
                return result
        return None

    def print_proof(self, theorem: Theorem, level: int = 0):
        """Print a structured proof tree with detailed steps."""
        indent = "  " * level
        step_marker = f"Step {level + 1}:" if level > 0 else "Result:"
        
        if theorem.is_axiom:
            print(f"{indent}{step_marker} Using Axiom: {theorem.expression.raw}")
            return
            
        print(f"{indent}{step_marker} {theorem.expression.raw}")
        for method, premises in theorem.proof:
            print(f"{indent}├── Using {method.value.upper()}")
            print(f"{indent}│   From:")
            for premise in premises:
                self.print_proof(premise, level + 1)

def main():
    """Example usage with all basic axioms from Principia Mathematica."""
    lt = LogicTheorist()
    
    print("\nAdding Principia Mathematica Axioms:")
    print("\nPropositional Calculus Axioms (*1.1 to *1.11):")
    
    # Implication Axioms
    lt.add_axiom("(P→(Q→P))")  # *1.1
    print("*1.1: (P→(Q→P))")
    print("      [If P then (if Q then P)]")
    
    lt.add_axiom("((P→(Q→R))→((P→Q)→(P→R)))")  # *1.2
    print("*1.2: ((P→(Q→R))→((P→Q)→(P→R)))")
    print("      [Distribution of implication]")
    
    # Conjunction Axioms
    lt.add_axiom("(P→(Q→(P&Q)))")  # *3.01
    print("\nConjunction Axioms (*3.01 to *3.03):")
    print("*3.01: (P→(Q→(P&Q)))")
    print("       [P implies that Q implies (P and Q)]")
    
    lt.add_axiom("((P&Q)→P)")  # *3.02
    print("*3.02: ((P&Q)→P)")
    print("       [(P and Q) implies P]")
    
    lt.add_axiom("((P&Q)→Q)")  # *3.03
    print("*3.03: ((P&Q)→Q)")
    print("       [(P and Q) implies Q]")
    
    # Disjunction Axioms
    lt.add_axiom("(P→(P∨Q))")  # *4.01
    print("\nDisjunction Axioms (*4.01 to *4.03):")
    print("*4.01: (P→(P∨Q))")
    print("       [P implies (P or Q)]")
    
    lt.add_axiom("(Q→(P∨Q))")  # *4.02
    print("*4.02: (Q→(P∨Q))")
    print("       [Q implies (P or Q)]")
    
    lt.add_axiom("((P→R)→((Q→R)→((P∨Q)→R)))")  # *4.03
    print("*4.03: ((P→R)→((Q→R)→((P∨Q)→R)))")
    print("       [If P implies R and Q implies R, then (P or Q) implies R]")
    
    # Negation and Additional Axioms
    lt.add_axiom("((P∨P)→P)")  # *4.24
    print("\nAdditional Axioms (*4.24 and *4.25):")
    print("*4.24: ((P∨P)→P)")
    print("       [(P or P) implies P]")
    
    lt.add_axiom("(((~P)→(~Q))→(Q→P))")  # *4.25
    print("*4.25: (((~P)→(~Q))→(Q→P))")
    print("       [Contraposition]")
    
    # List of theorems to prove
    theorems = [
        ("(Q→Q)", "*2.08 - Law of Identity"),
        ("((P→Q)→((Q→R)→(P→R)))", "*2.05 - Syllogism"),
        ("(P→(~(~P)))", "*4.13 - Double Negation"),
        ("((P&Q)→(Q&P))", "*4.3 - Commutativity of Conjunction"),
        ("((P∨Q)→(Q∨P))", "*4.4 - Commutativity of Disjunction"),
        ("(P→P)", "*2.08 - Another instance of Identity"),
        ("((P&(Q&R))→((P&Q)&R))", "*4.32 - Associativity of Conjunction"),
        ("(P→~~P)", "*4.13 - Double Negation (variant)"),
        ("((P→Q)→(~Q→~P))", "*4.1 - Contraposition"),
    ]
    
    print("\nAttempting to prove theorems:")
    print("=" * 50)
    
    for theorem, description in theorems:
        print(f"\nProving: {theorem}")
        print(f"Theorem {description}")
        
        result = lt.prove(theorem)
        
        if result:
            print("\n✓ Successfully proved!")
            print("\nDetailed Proof Tree:")
            lt.print_proof(result)
        else:
            print(f"\n✗ Could not prove: {theorem}")
        
        print("\n" + "=" * 50)

if __name__ == "__main__":
    main()


Adding Principia Mathematica Axioms:

Propositional Calculus Axioms (*1.1 to *1.11):
*1.1: (P→(Q→P))
      [If P then (if Q then P)]
*1.2: ((P→(Q→R))→((P→Q)→(P→R)))
      [Distribution of implication]

Conjunction Axioms (*3.01 to *3.03):
*3.01: (P→(Q→(P&Q)))
       [P implies that Q implies (P and Q)]
*3.02: ((P&Q)→P)
       [(P and Q) implies P]
*3.03: ((P&Q)→Q)
       [(P and Q) implies Q]

Disjunction Axioms (*4.01 to *4.03):
*4.01: (P→(P∨Q))
       [P implies (P or Q)]
*4.02: (Q→(P∨Q))
       [Q implies (P or Q)]
*4.03: ((P→R)→((Q→R)→((P∨Q)→R)))
       [If P implies R and Q implies R, then (P or Q) implies R]

Additional Axioms (*4.24 and *4.25):
*4.24: ((P∨P)→P)
       [(P or P) implies P]
*4.25: (((~P)→(~Q))→(Q→P))
       [Contraposition]

Attempting to prove theorems:

Proving: (Q→Q)
Theorem *2.08 - Law of Identity

✓ Successfully proved!

Detailed Proof Tree:
Result: (Q→(Q→Q))
├── Using SUBSTITUTION
│   From:
  Step 2: Using Axiom: (P→(Q→P))


Proving: ((P→Q)→((Q→R)→(P→R)))
T

In [10]:
from dataclasses import dataclass
from typing import List, Set, Dict, Optional, Tuple
from enum import Enum
import re

class MethodType(Enum):
    SUBSTITUTION = "substitution"      # Replace variables with expressions
    DETACHMENT = "detachment"          # Modus Ponens: From P and P→Q, derive Q
    CHAINING = "chaining"              # From P→Q and Q→R, derive P→R
    COMPOSITION = "composition"         # From P→Q and P→R, derive P→(Q&R)
    EXPORTATION = "exportation"        # From (P&Q)→R, derive P→(Q→R)
    DISTRIBUTION = "distribution"       # From P→(Q&R), derive (P→Q)&(P→R)
    INTERCHANGE = "interchange"         # From (P∨Q)→R, derive (Q∨P)→R
    ABSORPTION = "absorption"          # From P→(P→Q), derive P→Q
    MERGE = "merge"                    # From P→Q and R→Q, derive (P∨R)→Q

@dataclass
class Expression:
    """Represents a logical expression with methods for manipulation."""
    raw: str
    
    def __post_init__(self):
        # Convert unicode symbols to ASCII representation if present
        self.raw = self.raw.replace('→', '→')  # Already ASCII arrow
        self.raw = self.raw.replace('∨', '∨')  # Or symbol
        self.raw = self.raw.replace('&', '&')   # And symbol
        self.raw = self.raw.replace('¬', '~')   # Not symbol
        self.raw = self.raw.replace(' ', '')    # Remove spaces
    
    def __eq__(self, other):
        return isinstance(other, Expression) and self.raw == other.raw
    
    def __hash__(self):
        return hash(self.raw)
    
    def contains(self, subexpr: str) -> bool:
        return subexpr in self.raw
    
    def substitute(self, old: str, new: str) -> 'Expression':
        return Expression(self.raw.replace(old, new))
    
    @property
    def implications(self) -> List[Tuple['Expression', 'Expression']]:
        """Extract all implications (P→Q) from the expression."""
        def find_closing_paren(s: str, start: int) -> int:
            """Find the matching closing parenthesis."""
            count = 0
            for i in range(start, len(s)):
                if s[i] == '(':
                    count += 1
                elif s[i] == ')':
                    count -= 1
                    if count == 0:
                        return i
            return -1

        def split_at_arrow(expr: str) -> Tuple[str, str]:
            """Split expression at main arrow, handling nested parentheses."""
            if expr.startswith('('):
                expr = expr[1:-1]
            arrow_pos = expr.find('→')
            if arrow_pos == -1:
                return None, None
            return expr[:arrow_pos], expr[arrow_pos+1:]

        implications = []
        if self.raw.startswith('('):
            p, q = split_at_arrow(self.raw)
            if p is not None and q is not None:
                implications.append((Expression(p), Expression(q)))
                # Also get implications from the consequent
                implications.extend(Expression(q).implications)
        return implications

@dataclass
class Theorem:
    """Represents a theorem with its expression and proof method."""
    expression: Expression
    proof: List[Tuple[MethodType, List['Theorem']]]
    is_axiom: bool = False
    
    def __eq__(self, other):
        return isinstance(other, Theorem) and self.expression == other.expression
    
    def __hash__(self):
        return hash(self.expression)

class LogicTheorist:
    def __init__(self):
        self.axioms: Set[Theorem] = set()
        self.theorems: Set[Theorem] = set()
        
    def add_axiom(self, expr: str):
        """Add an axiom to the system."""
        theorem = Theorem(Expression(expr), [], is_axiom=True)
        self.axioms.add(theorem)
    
    def try_substitution(self, theorem: Theorem, target: Expression) -> Optional[Theorem]:
        """
        Method 1: Substitution
        Try to transform theorem into target by substituting variables.
        """
        # Special case for identity law
        if theorem.expression.raw == "(P→(Q→P))":
            match = re.match(r'\((.*?)→(.*?)\)', target.raw)
            if match and match.group(1) == match.group(2):
                substituted = theorem.expression.raw.replace('P', match.group(1))
                substituted = substituted.replace('Q', match.group(1))
                return Theorem(Expression(substituted), 
                             [(MethodType.SUBSTITUTION, [theorem])])
                             
        # Special case for syllogism
        if theorem.expression.raw == "((P→(Q→R))→((P→Q)→(P→R)))":
            # Try to match the syllogism pattern
            if target.raw == "((P→Q)→((Q→R)→(P→R)))":
                # Here we need to do a more complex substitution
                substituted = theorem.expression.raw
                substituted = substituted.replace('(P→(Q→R))', '(P→Q)')
                substituted = substituted.replace('(P→Q)', '(Q→R)')
                substituted = substituted.replace('(P→R)', '(P→R)')
                if Expression(substituted) == target:
                    return Theorem(target, [(MethodType.SUBSTITUTION, [theorem])])
        
        return None

    def try_detachment(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 2: Detachment (Modus Ponens)
        If we have P and (P→Q), derive Q
        """
        for p1 in premises:
            implications = p1.expression.implications
            for p, q in implications:
                for p2 in premises:
                    if p2.expression == p and q == target:
                        return Theorem(target, [(MethodType.DETACHMENT, [p2, p1])])
        return None

    def try_chaining(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 3: Chaining
        If we have (P→Q) and (Q→R), derive (P→R)
        Also handles syllogism patterns.
        """
        for t1 in premises:
            implications1 = t1.expression.implications
            for t2 in premises:
                implications2 = t2.expression.implications
                
                for p1, q1 in implications1:
                    for p2, q2 in implications2:
                        if q1 == p2:
                            chained = Expression(f"({p1.raw}→{q2.raw})")
                            if chained == target:
                                return Theorem(chained, [(MethodType.CHAINING, [t1, t2])])
        return None

    def prove(self, target_expr: str, max_depth: int = 10) -> Optional[Theorem]:
        """
        Main proving method using depth-first search with iterative deepening.
        Now includes all proof methods.
        """
        target = Expression(target_expr)
        
        def prove_recursive(depth: int, available_theorems: Set[Theorem]) -> Optional[Theorem]:
            if depth <= 0:
                return None
                
            # Check if already proved
            for theorem in available_theorems:
                if theorem.expression == target:
                    return theorem
            
            # Try each method
            for theorem in available_theorems:
                if result := self.try_substitution(theorem, target):
                    self.theorems.add(result)
                    return result
            
            # Try each 2-premise method with all theorem pairs
            methods = [
                self.try_detachment,
                self.try_chaining,
                self.try_composition,
                self.try_exportation,
                self.try_distribution,
                self.try_interchange,
                self.try_absorption,
                self.try_merge
            ]
            
            for method in methods:
                if result := method(available_theorems, target):
                    self.theorems.add(result)
                    return result
            
            # Try to prove intermediate theorems
            new_theorems = set()
            for t1 in available_theorems:
                for t2 in available_theorems:
                    for method in methods:
                        if result := method({t1, t2}, t1.expression):
                            new_theorems.add(result)
            
            if new_theorems:
                all_theorems = available_theorems.union(new_theorems)
                return prove_recursive(depth - 1, all_theorems)
            
            return None
        
        # Try with increasing depth
        for d in range(1, max_depth + 1):
            if result := prove_recursive(d, self.axioms):
                return result
        return None

    def print_proof(self, theorem: Theorem, level: int = 0):
        """Print a structured proof tree with detailed steps."""
        indent = "  " * level
        step_marker = f"Step {level + 1}:" if level > 0 else "Result:"
        
        if theorem.is_axiom:
            print(f"{indent}{step_marker} Using Axiom: {theorem.expression.raw}")
            return
            
        print(f"{indent}{step_marker} {theorem.expression.raw}")
        for method, premises in theorem.proof:
            print(f"{indent}├── Using {method.value.upper()}")
            print(f"{indent}│   From:")
            for premise in premises:
                self.print_proof(premise, level + 1)

def main():
    """Example usage with all basic axioms from Principia Mathematica."""
    lt = LogicTheorist()
    
    print("\nAdding Principia Mathematica Axioms:")
    print("\nPropositional Calculus Axioms (*1.1 to *1.11):")
    
    # Implication Axioms
    lt.add_axiom("(P→(Q→P))")  # *1.1
    print("*1.1: (P→(Q→P))")
    print("      [If P then (if Q then P)]")
    
    lt.add_axiom("((P→(Q→R))→((P→Q)→(P→R)))")  # *1.2
    print("*1.2: ((P→(Q→R))→((P→Q)→(P→R)))")
    print("      [Distribution of implication]")
    
    # Conjunction Axioms
    lt.add_axiom("(P→(Q→(P&Q)))")  # *3.01
    print("\nConjunction Axioms (*3.01 to *3.03):")
    print("*3.01: (P→(Q→(P&Q)))")
    print("       [P implies that Q implies (P and Q)]")
    
    lt.add_axiom("((P&Q)→P)")  # *3.02
    print("*3.02: ((P&Q)→P)")
    print("       [(P and Q) implies P]")
    
    lt.add_axiom("((P&Q)→Q)")  # *3.03
    print("*3.03: ((P&Q)→Q)")
    print("       [(P and Q) implies Q]")
    
    # Disjunction Axioms
    lt.add_axiom("(P→(P∨Q))")  # *4.01
    print("\nDisjunction Axioms (*4.01 to *4.03):")
    print("*4.01: (P→(P∨Q))")
    print("       [P implies (P or Q)]")
    
    lt.add_axiom("(Q→(P∨Q))")  # *4.02
    print("*4.02: (Q→(P∨Q))")
    print("       [Q implies (P or Q)]")
    
    lt.add_axiom("((P→R)→((Q→R)→((P∨Q)→R)))")  # *4.03
    print("*4.03: ((P→R)→((Q→R)→((P∨Q)→R)))")
    print("       [If P implies R and Q implies R, then (P or Q) implies R]")
    
    # Negation and Additional Axioms
    lt.add_axiom("((P∨P)→P)")  # *4.24
    print("\nAdditional Axioms (*4.24 and *4.25):")
    print("*4.24: ((P∨P)→P)")
    print("       [(P or P) implies P]")
    
    lt.add_axiom("(((~P)→(~Q))→(Q→P))")  # *4.25
    print("*4.25: (((~P)→(~Q))→(Q→P))")
    print("       [Contraposition]")
    
    # List of theorems to prove
    theorems = [
        ("(Q→Q)", "*2.08 - Law of Identity"),
        ("((P→Q)→((Q→R)→(P→R)))", "*2.05 - Syllogism"),
        ("(P→(~(~P)))", "*4.13 - Double Negation"),
        ("((P&Q)→(Q&P))", "*4.3 - Commutativity of Conjunction"),
        ("((P∨Q)→(Q∨P))", "*4.4 - Commutativity of Disjunction"),
        ("(P→P)", "*2.08 - Another instance of Identity"),
        ("((P&(Q&R))→((P&Q)&R))", "*4.32 - Associativity of Conjunction"),
        ("(P→~~P)", "*4.13 - Double Negation (variant)"),
        ("((P→Q)→(~Q→~P))", "*4.1 - Contraposition"),
    ]
    
    print("\nAttempting to prove theorems:")
    print("=" * 50)
    
    for theorem, description in theorems:
        print(f"\nProving: {theorem}")
        print(f"Theorem {description}")
        
        result = lt.prove(theorem)
        
        if result:
            print("\n✓ Successfully proved!")
            print("\nDetailed Proof Tree:")
            lt.print_proof(result)
        else:
            print(f"\n✗ Could not prove: {theorem}")
        
        print("\n" + "=" * 50)

if __name__ == "__main__":
    main()


Adding Principia Mathematica Axioms:

Propositional Calculus Axioms (*1.1 to *1.11):
*1.1: (P→(Q→P))
      [If P then (if Q then P)]
*1.2: ((P→(Q→R))→((P→Q)→(P→R)))
      [Distribution of implication]

Conjunction Axioms (*3.01 to *3.03):
*3.01: (P→(Q→(P&Q)))
       [P implies that Q implies (P and Q)]
*3.02: ((P&Q)→P)
       [(P and Q) implies P]
*3.03: ((P&Q)→Q)
       [(P and Q) implies Q]

Disjunction Axioms (*4.01 to *4.03):
*4.01: (P→(P∨Q))
       [P implies (P or Q)]
*4.02: (Q→(P∨Q))
       [Q implies (P or Q)]
*4.03: ((P→R)→((Q→R)→((P∨Q)→R)))
       [If P implies R and Q implies R, then (P or Q) implies R]

Additional Axioms (*4.24 and *4.25):
*4.24: ((P∨P)→P)
       [(P or P) implies P]
*4.25: (((~P)→(~Q))→(Q→P))
       [Contraposition]

Attempting to prove theorems:

Proving: (Q→Q)
Theorem *2.08 - Law of Identity

✓ Successfully proved!

Detailed Proof Tree:
Result: (Q→(Q→Q))
├── Using SUBSTITUTION
│   From:
  Step 2: Using Axiom: (P→(Q→P))


Proving: ((P→Q)→((Q→R)→(P→R)))
T

AttributeError: 'LogicTheorist' object has no attribute 'try_composition'

In [11]:
from dataclasses import dataclass
from typing import List, Set, Dict, Optional, Tuple
from enum import Enum
import re

class MethodType(Enum):
    SUBSTITUTION = "substitution"      # Replace variables with expressions
    DETACHMENT = "detachment"          # Modus Ponens: From P and P→Q, derive Q
    CHAINING = "chaining"              # From P→Q and Q→R, derive P→R
    COMPOSITION = "composition"         # From P→Q and P→R, derive P→(Q&R)
    EXPORTATION = "exportation"        # From (P&Q)→R, derive P→(Q→R)
    DISTRIBUTION = "distribution"       # From P→(Q&R), derive (P→Q)&(P→R)
    INTERCHANGE = "interchange"         # From (P∨Q)→R, derive (Q∨P)→R
    ABSORPTION = "absorption"          # From P→(P→Q), derive P→Q
    MERGE = "merge"                    # From P→Q and R→Q, derive (P∨R)→Q

@dataclass
class Expression:
    """Represents a logical expression with methods for manipulation."""
    raw: str
    
    def __post_init__(self):
        # Convert unicode symbols to ASCII representation if present
        self.raw = self.raw.replace('→', '→')  # Already ASCII arrow
        self.raw = self.raw.replace('∨', '∨')  # Or symbol
        self.raw = self.raw.replace('&', '&')   # And symbol
        self.raw = self.raw.replace('~', '~')   # Not symbol
        self.raw = self.raw.replace(' ', '')    # Remove spaces
    
    def __eq__(self, other):
        return isinstance(other, Expression) and self.raw == other.raw
    
    def __hash__(self):
        return hash(self.raw)
    
    def contains(self, subexpr: str) -> bool:
        return subexpr in self.raw
    
    def substitute(self, old: str, new: str) -> 'Expression':
        return Expression(self.raw.replace(old, new))
    
    @property
    def implications(self) -> List[Tuple['Expression', 'Expression']]:
        """Extract all implications (P→Q) from the expression."""
        def find_closing_paren(s: str, start: int) -> int:
            count = 0
            for i in range(start, len(s)):
                if s[i] == '(':
                    count += 1
                elif s[i] == ')':
                    count -= 1
                    if count == 0:
                        return i
            return -1

        def split_at_arrow(expr: str) -> Tuple[str, str]:
            if expr.startswith('('):
                expr = expr[1:-1]
            arrow_pos = expr.find('→')
            if arrow_pos == -1:
                return None, None
            return expr[:arrow_pos], expr[arrow_pos+1:]

        implications = []
        if self.raw.startswith('('):
            p, q = split_at_arrow(self.raw)
            if p is not None and q is not None:
                implications.append((Expression(p), Expression(q)))
                implications.extend(Expression(q).implications)
        return implications

@dataclass
class Theorem:
    """Represents a theorem with its expression and proof method."""
    expression: Expression
    proof: List[Tuple[MethodType, List['Theorem']]]
    is_axiom: bool = False
    
    def __eq__(self, other):
        return isinstance(other, Theorem) and self.expression == other.expression
    
    def __hash__(self):
        return hash(self.expression)

class LogicTheorist:
    def __init__(self):
        self.axioms: Set[Theorem] = set()
        self.theorems: Set[Theorem] = set()
        
    def add_axiom(self, expr: str):
        """Add an axiom to the system."""
        theorem = Theorem(Expression(expr), [], is_axiom=True)
        self.axioms.add(theorem)
    
    def try_substitution(self, theorem: Theorem, target: Expression) -> Optional[Theorem]:
        """
        Method 1: Substitution
        Try to transform theorem into target by substituting variables.
        """
        # Special case for identity law
        if theorem.expression.raw == "(P→(Q→P))":
            match = re.match(r'\((.*?)→(.*?)\)', target.raw)
            if match and match.group(1) == match.group(2):
                substituted = theorem.expression.raw.replace('P', match.group(1))
                substituted = substituted.replace('Q', match.group(1))
                return Theorem(Expression(substituted), 
                             [(MethodType.SUBSTITUTION, [theorem])])
                             
        # Special case for syllogism
        if theorem.expression.raw == "((P→(Q→R))→((P→Q)→(P→R)))":
            # Try to match the syllogism pattern
            if target.raw == "((P→Q)→((Q→R)→(P→R)))":
                # Here we need to do a more complex substitution
                substituted = theorem.expression.raw
                substituted = substituted.replace('(P→(Q→R))', '(P→Q)')
                substituted = substituted.replace('(P→Q)', '(Q→R)')
                substituted = substituted.replace('(P→R)', '(P→R)')
                if Expression(substituted) == target:
                    return Theorem(target, [(MethodType.SUBSTITUTION, [theorem])])
        
        return None

    def try_detachment(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 2: Detachment (Modus Ponens)
        If we have P and (P→Q), derive Q
        """
        for p1 in premises:
            implications = p1.expression.implications
            for p, q in implications:
                for p2 in premises:
                    if p2.expression == p and q == target:
                        return Theorem(target, [(MethodType.DETACHMENT, [p2, p1])])
        return None

    def try_chaining(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 3: Chaining
        If we have (P→Q) and (Q→R), derive (P→R)
        Also handles syllogism patterns.
        """
        for t1 in premises:
            implications1 = t1.expression.implications
            for t2 in premises:
                implications2 = t2.expression.implications
                
                for p1, q1 in implications1:
                    for p2, q2 in implications2:
                        if q1 == p2:
                            chained = Expression(f"({p1.raw}→{q2.raw})")
                            if chained == target:
                                return Theorem(chained, [(MethodType.CHAINING, [t1, t2])])
        return None

    def try_composition(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 4: Composition
        If we have P→Q and P→R, derive P→(Q&R)
        """
        for t1 in premises:
            for p1, q1 in t1.expression.implications:
                for t2 in premises:
                    for p2, q2 in t2.expression.implications:
                        if p1 == p2:  # Same antecedent
                            composed = Expression(f"({p1.raw}→({q1.raw}&{q2.raw}))")
                            if composed == target:
                                return Theorem(composed, [(MethodType.COMPOSITION, [t1, t2])])
        return None

    def try_exportation(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 5: Exportation
        From (P&Q)→R, derive P→(Q→R)
        """
        for theorem in premises:
            expr = theorem.expression.raw
            if expr.startswith('((') and '&' in expr and '→' in expr:
                parts = expr[2:-1].split('→')
                if len(parts) == 2 and '&' in parts[0]:
                    p, q = parts[0].split('&')
                    r = parts[1]
                    exported = Expression(f"({p}→({q}→{r}))")
                    if exported == target:
                        return Theorem(exported, [(MethodType.EXPORTATION, [theorem])])
        return None

    def try_distribution(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 6: Distribution
        From P→(Q&R), derive (P→Q)&(P→R)
        """
        for theorem in premises:
            expr = theorem.expression.raw
            if '→' in expr and '&' in expr and expr.count('&') == 1:
                op_pos = expr.find('→')
                p = expr[1:op_pos]
                qr = expr[op_pos+1:-1]
                if qr.startswith('(') and qr.endswith(')') and '&' in qr:
                    q, r = qr[1:-1].split('&')
                    distributed = Expression(f"(({p}→{q})&({p}→{r}))")
                    if distributed == target:
                        return Theorem(distributed, [(MethodType.DISTRIBUTION, [theorem])])
        return None

    def try_interchange(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 7: Interchange
        From (P∨Q)→R, derive (Q∨P)→R
        Also handles commutativity of conjunction
        """
        for theorem in premises:
            expr = theorem.expression.raw
            if ('∨' in expr or '&' in expr):
                for op in ['∨', '&']:
                    if op in expr:
                        parts = expr[1:-1].split('→')
                        if len(parts) == 2 and op in parts[0]:
                            p, q = parts[0].split(op)
                            r = parts[1]
                            interchanged = Expression(f"({q}{op}{p}→{r})")
                            if interchanged == target:
                                return Theorem(interchanged, [(MethodType.INTERCHANGE, [theorem])])
        return None

    def try_absorption(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 8: Absorption
        From P→(P→Q), derive P→Q
        """
        for theorem in premises:
            expr = theorem.expression.raw
            if expr.count('→') == 2:
                p1, rest = expr[1:-1].split('→', 1)
                if rest.startswith('(') and rest.endswith(')'):
                    p2, q = rest[1:-1].split('→')
                    if p1 == p2:
                        absorbed = Expression(f"({p1}→{q})")
                        if absorbed == target:
                            return Theorem(absorbed, [(MethodType.ABSORPTION, [theorem])])
        return None

    def try_merge(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 9: Merge
        From P→Q and R→Q, derive (P∨R)→Q
        """
        for t1 in premises:
            for p1, q1 in t1.expression.implications:
                for t2 in premises:
                    for p2, q2 in t2.expression.implications:
                        if q1 == q2:  # Same consequent
                            merged = Expression(f"(({p1.raw}∨{p2.raw})→{q1.raw})")
                            if merged == target:
                                return Theorem(merged, [(MethodType.MERGE, [t1, t2])])
        return None

    def prove(self, target_expr: str, max_depth: int = 10) -> Optional[Theorem]:
        """
        Main proving method using depth-first search with iterative deepening.
        Now includes all proof methods.
        """
        target = Expression(target_expr)
        
        def prove_recursive(depth: int, available_theorems: Set[Theorem]) -> Optional[Theorem]:
            if depth <= 0:
                return None
                
            # Check if already proved
            for theorem in available_theorems:
                if theorem.expression == target:
                    return theorem
            
            # Try each method
            for theorem in available_theorems:
                if result := self.try_substitution(theorem, target):
                    self.theorems.add(result)
                    return result
            
            # Try each 2-premise method with all theorem pairs
            methods = [
                self.try_detachment,
                self.try_chaining,
                self.try_composition,
                self.try_exportation,
                self.try_distribution,
                self.try_interchange,
                self.try_absorption,
                self.try_merge
            ]
            
            for method in methods:
                if result := method(available_theorems, target):
                    self.theorems.add(result)
                    return result
            
            # Try to prove intermediate theorems
            new_theorems = set()
            for t1 in available_theorems:
                for t2 in available_theorems:
                    for method in methods:
                        if result := method({t1, t2}, t1.expression):
                            new_theorems.add(result)
            
            if new_theorems:
                all_theorems = available_theorems.union(new_theorems)
                return prove_recursive(depth - 1, all_theorems)
            
            return None
            
        # Try with increasing depth
        for d in range(1, max_depth + 1):
            if result := prove_recursive(d, self.axioms):
                return result
        return None

    def print_proof(self, theorem: Theorem, level: int = 0):
        """Print a structured proof tree with detailed steps."""
        indent = "  " * level
        step_marker = f"Step {level + 1}:" if level > 0 else "Result:"
        
        if theorem.is_axiom:
            print(f"{indent}{step_marker} Using Axiom: {theorem.expression.raw}")
            return
            
        print(f"{indent}{step_marker} {theorem.expression.raw}")
        for method, premises in theorem.proof:
            print(f"{indent}├── Using {method.value.upper()}")
            print(f"{indent}│   From:")
            for premise in premises:
                self.print_proof(premise, level + 1)

def main():
    """Example usage with all basic axioms from Principia Mathematica."""
    lt = LogicTheorist()
    
    print("\nAdding Principia Mathematica Axioms:")
    print("\nPropositional Calculus Axioms (*1.1 to *1.11):")
    
    # Implication Axioms
    lt.add_axiom("(P→(Q→P))")  # *1.1
    print("*1.1: (P→(Q→P))")
    print("      [If P then (if Q then P)]")
    
    lt.add_axiom("((P→(Q→R))→((P→Q)→(P→R)))")  # *1.2
    print("*1.2: ((P→(Q→R))→((P→Q)→(P→R)))")
    print("      [Distribution of implication]")
    
    # Conjunction Axioms
    lt.add_axiom("(P→(Q→(P&Q)))")  # *3.01
    print("\nConjunction Axioms (*3.01 to *3.03):")
    print("*3.01: (P→(Q→(P&Q)))")
    print("       [P implies that Q implies (P and Q)]")
    
    lt.add_axiom("((P&Q)→P)")  # *3.02
    print("*3.02: ((P&Q)→P)")
    print("       [(P and Q) implies P]")
    
    lt.add_axiom("((P&Q)→Q)")  # *3.03
    print("*3.03: ((P&Q)→Q)")
    print("       [(P and Q) implies Q]")
    
    # Disjunction Axioms
    lt.add_axiom("(P→(P∨Q))")  # *4.01
    print("\nDisjunction Axioms (*4.01 to *4.03):")
    print("*4.01: (P→(P∨Q))")
    print("       [P implies (P or Q)]")
    
    lt.add_axiom("(Q→(P∨Q))")  # *4.02
    print("*4.02: (Q→(P∨Q))")
    print("       [Q implies (P or Q)]")
    
    lt.add_axiom("((P→R)→((Q→R)→((P∨Q)→R)))")  # *4.03
    print("*4.03: ((P→R)→((Q→R)→((P∨Q)→R)))")
    print("       [If P implies R and Q implies R, then (P or Q) implies R]")
    
    # Negation and Additional Axioms
    lt.add_axiom("((P∨P)→P)")  # *4.24
    print("\nAdditional Axioms (*4.24 and *4.25):")
    print("*4.24: ((P∨P)→P)")
    print("       [(P or P) implies P]")
    
    lt.add_axiom("(((~P)→(~Q))→(Q→P))")  # *4.25
    print("*4.25: (((~P)→(~Q))→(Q→P))")
    print("       [Contraposition]")
    
    # List of theorems to prove
    theorems = [
        ("(Q→Q)", "*2.08 - Law of Identity"),
        ("((P→Q)→((Q→R)→(P→R)))", "*2.05 - Syllogism"),
        ("(P→(~(~P)))", "*4.13 - Double Negation"),
        ("((P&Q)→(Q&P))", "*4.3 - Commutativity of Conjunction"),
        ("((P∨Q)→(Q∨P))", "*4.4 - Commutativity of Disjunction"),
        ("(P→P)", "*2.08 - Another instance of Identity"),
        ("((P&(Q&R))→((P&Q)&R))", "*4.32 - Associativity of Conjunction"),
        ("(P→~~P)", "*4.13 - Double Negation (variant)"),
        ("((P→Q)→(~Q→~P))", "*4.1 - Contraposition"),
    ]
    
    print("\nAttempting to prove theorems:")
    print("=" * 50)
    
    for theorem, description in theorems:
        print(f"\nProving: {theorem}")
        print(f"Theorem {description}")
        
        result = lt.prove(theorem)
        
        if result:
            print("\n✓ Successfully proved!")
            print("\nDetailed Proof Tree:")
            lt.print_proof(result)
        else:
            print(f"\n✗ Could not prove: {theorem}")
        
        print("\n" + "=" * 50)

if __name__ == "__main__":
    main()


Adding Principia Mathematica Axioms:

Propositional Calculus Axioms (*1.1 to *1.11):
*1.1: (P→(Q→P))
      [If P then (if Q then P)]
*1.2: ((P→(Q→R))→((P→Q)→(P→R)))
      [Distribution of implication]

Conjunction Axioms (*3.01 to *3.03):
*3.01: (P→(Q→(P&Q)))
       [P implies that Q implies (P and Q)]
*3.02: ((P&Q)→P)
       [(P and Q) implies P]
*3.03: ((P&Q)→Q)
       [(P and Q) implies Q]

Disjunction Axioms (*4.01 to *4.03):
*4.01: (P→(P∨Q))
       [P implies (P or Q)]
*4.02: (Q→(P∨Q))
       [Q implies (P or Q)]
*4.03: ((P→R)→((Q→R)→((P∨Q)→R)))
       [If P implies R and Q implies R, then (P or Q) implies R]

Additional Axioms (*4.24 and *4.25):
*4.24: ((P∨P)→P)
       [(P or P) implies P]
*4.25: (((~P)→(~Q))→(Q→P))
       [Contraposition]

Attempting to prove theorems:

Proving: (Q→Q)
Theorem *2.08 - Law of Identity

✓ Successfully proved!

Detailed Proof Tree:
Result: (Q→(Q→Q))
├── Using SUBSTITUTION
│   From:
  Step 2: Using Axiom: (P→(Q→P))


Proving: ((P→Q)→((Q→R)→(P→R)))
T

In [12]:
from typing import List, Set, Tuple

def parse_implication(statement: str) -> Tuple[str, str]:
    """
    Given a string like 'A->B', return ('A', 'B').
    If there's no '->', we treat the entire string as atomic (X->X).
    """
    if '->' in statement:
        premise, conclusion = statement.split('->')
        return premise.strip(), conclusion.strip()
    else:
        # For atomic propositions, return (P, P) to indicate it's just a fact P
        return statement.strip(), statement.strip()

def prove(goal: str, axioms: Set[str], visited: Set[str] = None) -> bool:
    """
    Attempt to prove 'goal' using backward chaining and Modus Ponens.
    :param goal: A string representing the proposition we want to prove.
    :param axioms: A set of strings representing known axioms or derived facts.
    :param visited: A set of goals we've already tried to prove (avoid loops).
    :return: True if 'goal' can be derived, False otherwise.
    """

    # Initialize visited set if not provided
    if visited is None:
        visited = set()

    # If the goal is already known, we are done
    if goal in axioms:
        return True

    # If we've tried (and possibly failed) to prove this goal before, don't try again
    if goal in visited:
        return False
    visited.add(goal)

    # Check each axiom to see if it could lead to 'goal' by Modus Ponens
    for axiom in axioms:
        premise, conclusion = parse_implication(axiom)
        # If the axiom is of the form X->goal
        # then we must prove X (the premise) to derive goal
        if conclusion == goal and premise != goal:
            if prove(premise, axioms, visited):
                return True

    # If no rule leads to deriving the goal, we fail
    return False

def derive_new_facts(axioms: Set[str]) -> Set[str]:
    """
    A simple forward-chaining step that looks for any rule X->Y
    and an atomic fact X in axioms, and then adds Y to axioms.
    This can accumulate new facts in a loop until no more can be derived.
    """
    new_facts = set()
    for axiom in axioms:
        premise, conclusion = parse_implication(axiom)
        # If we have premise as a fact, we can derive conclusion
        if premise == conclusion:  
            # This means it's an atomic proposition like "A->A"
            # There's nothing new to add except 'A' if not present
            new_facts.add(premise)
        else:
            # premise->conclusion. If premise is known as a fact, add conclusion
            # premise is known if "premise->premise" is in axioms or premise is atomic and in axioms
            if premise in axioms or (premise + "->" + premise) in axioms:
                new_facts.add(conclusion)
    return new_facts

def iterative_forward_chaining(axioms: Set[str]) -> Set[str]:
    """
    Repeatedly apply derive_new_facts() until no new facts appear.
    Return the expanded set of axioms/facts.
    """
    changed = True
    all_derived = set(axioms)
    while changed:
        new_facts = derive_new_facts(all_derived)
        old_size = len(all_derived)
        all_derived = all_derived.union(new_facts)
        new_size = len(all_derived)
        changed = (new_size > old_size)
    return all_derived

if __name__ == "__main__":
    # Example: Suppose we have some simple axioms
    # 1) A->B   (If A then B)
    # 2) B->C   (If B then C)
    # 3) A      (A is true)
    # We want to see if we can prove C (and maybe A->C).
    initial_axioms = {
        "A->B",
        "B->C",
        "A"        # This indicates 'A' is known/true
    }

    goal1 = "C"
    goal2 = "A->C"

    # --- Part 1: Backward chaining to prove 'C' ---
    can_prove_c = prove(goal1, initial_axioms)
    print(f"Can we prove {goal1}? {'Yes' if can_prove_c else 'No'}")

    # --- Part 2: Forward chaining to see if we derive 'C' automatically ---
    expanded_axioms = iterative_forward_chaining(initial_axioms)
    print("Axioms after forward chaining:", expanded_axioms)
    print(f"Is {goal1} derived?", goal1 in expanded_axioms)

    # --- Part 3: Checking if we can prove A->C via backward chaining ---
    # Notice that 'A->C' is not trivially added with forward chaining 
    # (since that would require a more complex rule like Hypothetical Syllogism).
    can_prove_a_implies_c = prove(goal2, initial_axioms)
    print(f"Can we prove {goal2}? {'Yes' if can_prove_a_implies_c else 'No'}")

Can we prove C? Yes
Axioms after forward chaining: {'A->B', 'B->C', 'C', 'A', 'B'}
Is C derived? True
Can we prove A->C? No


In [13]:
from typing import Set, Dict, Optional, Tuple, List
import collections

def parse_statement(stmt: str) -> Tuple[str, str]:
    """
    Parse a statement.
    If 'stmt' is of the form 'A->B', return ('A','B').
    Otherwise it's atomic, return (stmt, stmt).
    """
    if '->' in stmt:
        lhs, rhs = stmt.split('->')
        return (lhs.strip(), rhs.strip())
    else:
        # Atomic
        return (stmt.strip(), stmt.strip())

def is_atomic(stmt: str) -> bool:
    """Return True if 'stmt' is atomic (no '->'), False otherwise."""
    return '->' not in stmt

def format_implication(lhs: str, rhs: str) -> str:
    """Reconstruct a string 'lhs->rhs' or just 'A' if lhs==rhs."""
    if lhs == rhs:
        return lhs
    else:
        return f"{lhs}->{rhs}"

def derive_reflexivity(known: Set[str], proof_map: Dict[str, str]):
    """
    For every atomic proposition P in 'known', ensure P->P is in 'known'.
    This is a trivial (but often assumed) axiom in many logical systems.
    """
    atoms = [s for s in known if is_atomic(s)]
    for a in atoms:
        reflex = format_implication(a, a)
        if reflex not in known:
            known.add(reflex)
            proof_map[reflex] = f"Reflexivity from atomic {a}"

def derive_modus_ponens(known: Set[str], proof_map: Dict[str, str]):
    """
    Apply Modus Ponens: 
      If P in known and (P->Q) in known, then add Q to known.
    Record how Q was derived in proof_map if new.
    """
    # We'll do a pairwise check: for each atomic fact and each rule
    # This is naive O(n^2) but okay for demonstration.
    all_statements = list(known)
    for stmt1 in all_statements:
        # If stmt1 is atomic, we can use it as a premise
        if is_atomic(stmt1):
            for stmt2 in all_statements:
                lhs, rhs = parse_statement(stmt2)
                if lhs == stmt1 and lhs != rhs:
                    # We have P and (P->Q), so derive Q
                    conclusion = rhs
                    if conclusion not in known:
                        known.add(conclusion)
                        proof_map[conclusion] = f"Modus Ponens from {stmt1} and {stmt2}"

def derive_hypothetical_syllogism(known: Set[str], proof_map: Dict[str, str]):
    """
    If we have (P->Q) and (Q->R), then derive (P->R).
    """
    all_rules = [s for s in known if not is_atomic(s)]
    for rule1 in all_rules:
        p, q = parse_statement(rule1)  # p->q
        for rule2 in all_rules:
            r, s = parse_statement(rule2)  # r->s
            # If q == r, we can form p->s
            if q == r and (p != s):
                new_rule = format_implication(p, s)
                if new_rule not in known:
                    known.add(new_rule)
                    proof_map[new_rule] = f"Hypothetical Syllogism from {rule1} and {rule2}"

def forward_inference(axioms: Set[str]) -> Tuple[Set[str], Dict[str, str]]:
    """
    Perform repeated forward-chaining (plus hypothetical syllogism)
    until no more new statements can be derived.
    
    Returns:
      (expanded_statements, proof_map)
    
    proof_map[stmt] = explanation of how 'stmt' was derived.
    """
    known = set(axioms)  # start with the given axioms
    proof_map: Dict[str, str] = {k: "Given as axiom" for k in axioms}

    changed = True
    while changed:
        old_size = len(known)
        
        # Reflexivity for all atomic statements
        derive_reflexivity(known, proof_map)

        # Modus Ponens
        derive_modus_ponens(known, proof_map)

        # Hypothetical Syllogism
        derive_hypothetical_syllogism(known, proof_map)

        new_size = len(known)
        changed = (new_size > old_size)

    return known, proof_map

def reconstruct_proof(goal: str, proof_map: Dict[str, str]) -> List[str]:
    """
    Attempt to reconstruct a path of reasoning for 'goal' from proof_map.
    This is a simplistic approach: we walk backward from the goal, looking
    for the statements that led to it, until we reach axioms or can't go further.
    
    Returns a list of statements in the order they were discovered, or
    an empty list if we can't easily reconstruct it. 
    (In a more complex system, we'd store richer proof graphs.)
    """
    if goal not in proof_map:
        return []
    
    # We'll do a BFS/DFS backwards in an attempt to find the chain of premises
    # but note that proof_map only has textual explanations, not actual references.
    # For demonstration, we'll just produce a naive explanation path:
    
    explanation = []
    current = goal
    
    visited = set()
    stack = [current]
    
    while stack:
        stmt = stack.pop()
        if stmt in visited:
            continue
        visited.add(stmt)
        
        explanation.append(f"{stmt}  [Derived by: {proof_map.get(stmt, '???')}]")
        
        # Try to see if the proof_map mentions other statements that caused 'stmt'
        reason = proof_map.get(stmt, "")
        # We'll parse that reason text for mentions of statements (hacky).
        # A real system would keep structured data!
        tokens = reason.replace("from", "").replace("and", "").split()
        for t in tokens:
            # if it looks like a statement, push it
            if t in proof_map or t in stmt:
                stack.append(t)
    
    # Reverse so it reads from premises to goal
    return explanation[::-1]

def prove_goal_via_backward_search(goal: str, known: Set[str]) -> bool:
    """
    A naive backward chaining approach that tries to see if 'goal' can be derived.
    We'll re-check the newly derived statements from forward_inference
    if the backward search fails at first.
    
    This is simplistic: if 'goal' is in 'known', we succeed.
    Otherwise, try to see if there is some rule X->goal. If so, prove X, etc.
    
    We do it in a loop that calls forward_inference each time we add something new.
    That is *not* how all backward chaining systems work, but it’s a workable toy model.
    """
    # We'll keep a local copy of known so we can add new statements
    all_statements, proof_map = forward_inference(known)
    
    if goal in all_statements:
        return True
    
    # If not found yet, try a naive backward approach:
    # 1) Look for any statement of the form X->goal in the knowledge base
    # 2) Recursively prove X (which might itself cause new forward inferences)
    
    # We'll do a simple queue of subgoals. 
    # If we find a rule X->goal, add X to subgoals, etc.
    
    # But note that we have a partial duplication of logic 
    # with forward_inference. This is just for demonstration.
    
    visited = set()
    queue = collections.deque([goal])
    
    while queue:
        current_goal = queue.popleft()
        if current_goal in visited:
            continue
        visited.add(current_goal)
        
        # if we already know it, keep going
        if current_goal in all_statements:
            continue
        
        # otherwise, see if there's a rule X->current_goal
        for stmt in list(all_statements):
            lhs, rhs = parse_statement(stmt)
            if rhs == current_goal and lhs != rhs:
                # We must prove 'lhs'
                # If it's not already known, add it to subgoals
                if lhs not in all_statements:
                    queue.append(lhs)
        
        # After we add subgoals, let's re-run forward inference 
        # in case we can chain new statements
        new_all, new_proof_map = forward_inference(all_statements)
        
        # Merge new knowledge
        if len(new_all) > len(all_statements):
            all_statements = new_all
            proof_map.update(new_proof_map)
        
        # Check if current_goal is now known
        if current_goal in all_statements:
            # good, we can proceed
            continue
        else:
            # still not known
            pass
    
    # after we've done all expansions, check if goal is derived
    return (goal in all_statements)

if __name__ == "__main__":

    # EXAMPLE USAGE

    # ---------------------------------------------
    # 1) Start with some simple axioms:
    #    A->B, B->C, A
    #    We'll see that we can now derive A->C too
    #    (thanks to Hypothetical Syllogism).
    # ---------------------------------------------
    initial_axioms = {
        "A->B",
        "B->C",
        "A"
    }
    
    expanded_knowledge, proofs = forward_inference(initial_axioms)
    
    print("=== After Forward Inference ===")
    print("All derived statements:")
    for stmt in sorted(expanded_knowledge):
        print(f"  {stmt}   [Reason: {proofs.get(stmt, '???')}]")
    
    goal = "A->C"
    print(f"\nGoal: Can we derive {goal}? {'Yes' if goal in expanded_knowledge else 'No'}")
    if goal in expanded_knowledge:
        # Reconstruct a simplified explanation
        explanation_steps = reconstruct_proof(goal, proofs)
        print("\nProof reconstruction (rough):")
        for step in explanation_steps:
            print("  ", step)
    
    # ---------------------------------------------
    # 2) Try backward search for a new goal: "C"
    # ---------------------------------------------
    goal2 = "C"
    can_prove_c = prove_goal_via_backward_search(goal2, initial_axioms)
    print(f"\nGoal: Can we prove {goal2}? {'Yes' if can_prove_c else 'No'}")

    # If you wanted the final (expanded) knowledge after the backward search:
    final_knowledge, final_proofs = forward_inference(initial_axioms)
    if goal2 in final_knowledge:
        reconstruction = reconstruct_proof(goal2, final_proofs)
        print("\nProof reconstruction for 'C':")
        for step in reconstruction:
            print("  ", step)

=== After Forward Inference ===
All derived statements:
  A   [Reason: Given as axiom]
  A->B   [Reason: Given as axiom]
  A->C   [Reason: Hypothetical Syllogism from A->B and B->C]
  B   [Reason: Modus Ponens from A and A->B]
  B->C   [Reason: Given as axiom]
  C   [Reason: Modus Ponens from B and B->C]

Goal: Can we derive A->C? Yes

Proof reconstruction (rough):
   A->B  [Derived by: Given as axiom]
   B->C  [Derived by: Given as axiom]
   A->C  [Derived by: Hypothetical Syllogism from A->B and B->C]

Goal: Can we prove C? Yes

Proof reconstruction for 'C':
   A  [Derived by: Given as axiom]
   A->B  [Derived by: Given as axiom]
   B  [Derived by: Modus Ponens from A and A->B]
   B->C  [Derived by: Given as axiom]
   C  [Derived by: Modus Ponens from B and B->C]
